In [76]:
# import libraries 

import yfinance as yf 
import numpy as np 
import pandas as pd 

# download data from yfinance dataset 
def fetch_and_process_hourly_data(ticker, start_date, end_date): 

    # Download hourly stock data with interval hardcoded into the function as 1h
    df = yf.download(ticker, start= start_date, end = end_date, interval= "1h")

    # Check if data is multi-index columns 
    if isinstance(df.columns, pd.MultiIndex): 
        # Extract ticker's data by slicing the second level of columns
        df_ticker = df.xs(ticker, axis = 1, level = 1)
    else: 
        df_ticker = df
    
    # Compute log returns on Close price 
    df_ticker["log_return"] = np.log(df_ticker["Close"] / df_ticker["Close"].shift(1))

    # Remove the first row after computing log returns to remove NaN log return 
    df_ticker = df_ticker.dropna(subset = ['log_return'])

    print(df_ticker)
    

In [77]:
# Example usage: 

ticker = "AAPL"
start_date = "2025-07-25"
end_date = "2025-07-30"

data = fetch_and_process_hourly_data(ticker, start_date, end_date)

/var/folders/34/mh55f40j11qcbq5hh0hfw_3w0000gn/T/ipykernel_84007/4039574915.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start= start_date, end = end_date, interval= "1h")
[*********************100%***********************]  1 of 1 completed

Price                           Close        High         Low        Open  \
Datetime                                                                    
2025-07-25 14:30:00+00:00  214.509995  215.240005  214.210007  214.589996   
2025-07-25 15:30:00+00:00  214.389603  214.639999  214.050003  214.500000   
2025-07-25 16:30:00+00:00  214.169998  214.589996  214.039993  214.389999   
2025-07-25 17:30:00+00:00  213.889999  214.509995  213.820007  214.170395   
2025-07-25 18:30:00+00:00  213.500000  214.009995  213.399994  213.910004   
2025-07-25 19:30:00+00:00  213.960007  214.000000  213.490005  213.505005   
2025-07-28 13:30:00+00:00  214.520004  214.740005  213.580002  214.029999   
2025-07-28 14:30:00+00:00  214.434998  214.845001  214.122299  214.535004   
2025-07-28 15:30:00+00:00  214.539993  214.750000  214.289993  214.440002   
2025-07-28 16:30:00+00:00  213.645004  214.574997  213.414993  214.535004   
2025-07-28 17:30:00+00:00  213.800003  214.020004  213.619995  213.634995   